# Taller Flujo de Potencia con Newton-Raphson

In [5]:
import numpy as np
from numpy import sin, cos, pi

In [11]:
z12 = 0.009+0.035j
z13 = 0.01+0.04j
z23 = 0.008 + 0.03j
y12 = 1/z12
y13 = 1/z13
y21 = y12
y23 = 1/z23
y31 = y13
y32 = y23

Y = np.array([[y12+y13, -y12, -y13],
              [-y21, y21+y23, -y23],
              [-y31, -y32, y31+y32]])

P2sch = 4.2
P3sch = -6.1
Q3sch = -3
v1, v2, v3 = 1.01, 1.03, 1 # Tensiones iniciales
ang1, ang2, ang3 = 0, 0, 0 # Ángulos iniciales

v = np.array([v1, v2, v3])
ang = np.array([ang1, ang2, ang3])

In [33]:
# Funciones para Newton-Raphson
def P(v,Y,i):
    return v[i-1]*sum([abs(Y[i-1,k])*v[k]*cos(np.angle(Y[i-1,k])-ang[i-1]+ang[k]) for k in range(len(v))])

def Q(v,Y,i):
    return -v[i-1]*sum([abs(Y[i-1,k])*v[k]*sin(np.angle(Y[i-1,k])-ang[i-1]+ang[k]) for k in range(len(v))])

def dPdD(v,Y,i,k):
    return v[i-1]*sum([v[k-1]*abs(Y[i-1,k-1])*sin(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1]) for k in range(1,4) if k != i]) if i==k \
        else -v[i-1]*v[k-1]*abs(Y[i-1,k-1])*sin(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1])

def dPdV(v,Y,i,k):
    return 2*v[i-1]*abs(Y[i-1,k-1])*cos(np.angle(Y[i-1,k-1]))+sum([v[k-1]*abs(Y[i-1,k-1])*cos(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1]) for k in range(1,4) if k != i]) if i==k \
        else v[i-1]*v[k-1]*abs(Y[i-1,k-1])*cos(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1])

def dQdD(v,Y,i,k):
    return v[i-1]*sum([v[k-1]*abs(Y[i-1,k-1])*cos(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1]) for k in range(1,4) if k != i]) if i==k \
        else -v[i-1]*v[k-1]*abs(Y[i-1,k-1])*cos(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1])
    
def dQdV(v,Y,i,k):
    return -2*v[i-1]*abs(Y[i-1,k-1])*sin(np.angle(Y[i-1,k-1]))-sum([v[k-1]*abs(Y[i-1,k-1])*sin(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1]) for k in range(1,4) if k != i]) if i==k \
        else -v[i-1]*v[k-1]*abs(Y[i-1,k-1])*sin(np.angle(Y[i-1,k-1])-ang[i-1]+ang[k-1])

## Punto 1

In [12]:
P2, P3, Q3 = P(v,Y,2), P(v,Y,3), Q(v,Y,3)
delP2 = P2sch - P2
delP3 = P3sch - P3
delQ3 = Q3sch - Q3
deltaPQ = np.array([delP2, delP3, delQ3])
error = max(abs(deltaPQ))

print("Iteración: 0")
print("P2:",P2)
print("P3:",P3)
print("Q3:",Q3)
print("ángulo v2:", ang2*180/pi)
print("ángulo v3:", ang3*180/pi)
print("voltaje v3:",v3)
print("Error:",error)
print()
tol = 0.001
i = 0

while error > tol:
    i+=1
    J = np.array([[dPdD(v,Y,2,2), dPdD(v,Y,2,3), dPdV(v,Y,2,3)],
                  [dPdD(v,Y,3,2), dPdD(v,Y,3,3), dPdV(v,Y,3,3)],
                  [dQdD(v,Y,3,2), dQdD(v,Y,3,3), dQdV(v,Y,3,3)]])

    delta_x = np.matmul(np.linalg.inv(J),deltaPQ)
    delta2 = delta_x[0]
    delta3 = delta_x[1]
    deltav3 = delta_x[2]
    ang2 += delta2
    ang3 += delta3
    v3 += deltav3
    
    v = np.array([v1, v2, v3])
    ang = np.array([ang1, ang2, ang3])
    
    P2, P3, Q3 = P(v,Y,2), P(v,Y,3), Q(v,Y,3)
    delP2 = P2sch - P2
    delP3 = P3sch - P3
    delQ3 = Q3sch - Q3
    deltaPQ = np.array([delP2, delP3, delQ3])
    error = max(abs(deltaPQ))
    print("Iteración:",i)
    print("P2:",P2)
    print("P3:",P3)
    print("Q3:",Q3)
    print("ángulo v2:", ang2*180/pi)
    print("ángulo v3:", ang3*180/pi)
    print("voltaje v3:",v3)
    print("Error:",error)
    print()
    

Iteración: 0
P2: 0.39839171903694187
P3: -0.3077861850134198
Q3: -1.1689040761532823
ángulo v2: 0.0
ángulo v3: 0.0
voltaje v3: 1
Error: 5.79221381498658

Iteración: 1
P2: 4.055904404313596
P3: -5.699377772717299
Q3: -2.6807520882758045
ángulo v2: 0.6091014012030788
ángulo v3: -4.763220970243675
voltaje v3: 0.941540122267472
Error: 0.40062222728270047

Iteración: 2
P2: 4.201389057538321
P3: -6.095245409171732
Q3: -2.995574275296306
ángulo v2: 0.4636292578609864
ángulo v3: -5.221825466396367
voltaje v3: 0.9325297675812809
Error: 0.004754590828267347

Iteración: 3
P2: 4.200043797670946
P3: -6.099999194782361
Q3: -2.999999145580604
ángulo v2: 0.4575095645807756
ángulo v3: -5.229799594890539
voltaje v3: 0.9324077394213218
Error: 4.379767094597753e-05



In [15]:
P1, Q1, Q2 = P(v,Y,1), Q(v,Y,1), Q(v,Y,2)
print("Resultados NR")
print("Número de iteraciones:",i)
print("Voltaje V1:",v1)
print("Voltaje V2:",v2)
print("Voltaje V3:",v3)
print("Ángulo V1:",ang1*180/pi)
print("Ángulo V2:",ang2*180/pi)
print("Ángulo V3:",ang3*180/pi)
print("P1:",P1)
print("Q1:",Q1)
print("P2:",P2)
print("Q2:",Q2)
print("P3:",P3)
print("Q3:",Q3)
print("Error:",error)

Resultados NR
Número de iteraciones: 3
Voltaje V1: 1.01
Voltaje V2: 1.03
Voltaje V3: 0.9324077394213218
Ángulo V1: 0.0
Ángulo V2: 0.4575095645807756
Ángulo V3: -5.229799594890539
P1: 2.142208306136961
Q1: 0.9480477907764978
P2: 4.200043797670946
Q2: 2.98123018782394
P3: -6.099999194782361
Q3: -2.999999145580604
Error: 4.379767094597753e-05


In [18]:
e = v * np.exp(1j*ang)

print("Tensiones nodales")
for i in e:
    print("{:.4} ∠ {:.5}°".format(abs(i),np.angle(i)*180/pi))
print()

Tensiones nodales
1.01 ∠ 0.0°
1.03 ∠ 0.45751°
0.9324 ∠ -5.2298°



## Punto 2

In [34]:
e1, e2, e3 = e
I12 = (e1-e2)*y12;
I13 = (e1-e3)*y13;
I21 = (e2-e1)*y21;
I23 = (e2-e3)*y23;
I31 = (e3-e1)*y31;
I32 = (e3-e2)*y32;

S12 = e1*I12.conjugate()
S21 = e2*I21.conjugate()
S13 = e1*I13.conjugate()
S31 = e3*I31.conjugate()
S23 = e2*I23.conjugate()
S32 = e3*I32.conjugate()

SG_1 = 3+1j + S12 + S13
SG_2 = 2+0.5j + S21 + S23

print("Potencias generadas")
print("Generador 1: {:.4f}".format(SG_1))
print("Generador 2: {:.4f}".format(SG_2))
print()

Potencias generadas
Generador 1: 5.1422+1.9480j
Generador 2: 6.2000+3.4812j



## Punto 3

In [32]:
# Potencia de pérdidas
SL = sum(e * np.conjugate(Y @ e))
print("Potencia de pérdidas: {:.4f}".format(SL))

Potencia de pérdidas: 0.2423+0.9293j
